In [1]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
from Heart_Segmentation.utils.common import read_yaml , create_directories , save_json
from Heart_Segmentation.constants import *
import optuna
from dataclasses import dataclass


In [2]:
%pwd

'/home/priyanshu1303d/Projects/Heart_Segmentation/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/priyanshu1303d/Projects/Heart_Segmentation'

In [5]:
@dataclass(frozen= True)
class ModelOptimizationConfig:
    root_dir: Path
    images_dir: Path
    labels_dir: Path
    model_save_path: Path
    model_metrics_json: Path
    best_model_path: Path
    params_epochs: int
    params_learning_rate: float
    params_batch_size: int
    params_weight_decay: float
    params_dropout_rate: float
    params_optimizer_name: str

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_optimization_config(self, trial) -> ModelOptimizationConfig:
        config = self.config.model_optimization
        create_directories([config.root_dir])

        # Get hyperparameters from Optuna trial
        model_optimization_config = ModelOptimizationConfig(
            root_dir=Path(config.root_dir),
            images_dir=Path(config.images_dir),
            labels_dir=Path(config.labels_dir),
            model_save_path=Path(config.model_save_path),
            model_metrics_json=Path(config.model_metrics_json),
            best_model_path=Path(config.best_model_path),
            params_epochs=trial.suggest_int('epochs', 5, 10),
            params_learning_rate=trial.suggest_float('lr', 1e-5, 1e-3, log=True),
            params_batch_size=trial.suggest_categorical('batch_size', [16, 32, 64, 128]),
            params_weight_decay=trial.suggest_float('weight_decay', 1e-5, 1e-1, log=True),
            params_dropout_rate=trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1),
            params_optimizer_name=trial.suggest_categorical('Optimizer_name', ['Adam', 'SGD', 'RMSprop'])
        )
        return model_optimization_config

In [7]:
from torch.utils.data import Dataset , DataLoader
from Heart_Segmentation import logger
import nibabel as nib
import torch
import torch.nn as nn

In [8]:
class HeartSegmentationDataset(Dataset):
    def __init__(self, images_dir: str, labels_dir: str):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.image_files = [f for f in os.listdir(images_dir) if f.endswith('.nii.gz')]
        logger.info(f"Found {len(self.image_files)} files in {images_dir}")
        if not self.image_files:
            raise ValueError(f"No .nii.gz files found in {images_dir}")

    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        image_path = os.path.join(self.images_dir, image_file)
        label_path = os.path.join(self.labels_dir, image_file)
        
        image = nib.load(image_path).get_fdata()
        label = nib.load(label_path).get_fdata()
        
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # [1, 128, 128, 64]
        label = torch.tensor(label, dtype=torch.long)  # [128, 128, 64]
        
        return image, label, image_file

In [9]:
from monai.networks.nets import UNet

In [ ]:
class ModelOptimization:
    def __init__(self, config: ModelOptimizationConfig):
        self.config = config
        #### self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # DataLoader initialization
        train_dataset = HeartSegmentationDataset(self.config.images_dir, self.config.labels_dir)
        self.train_loader = DataLoader(train_dataset, batch_size=self.config.params_batch_size, shuffle=True, pin_memory=True)
        self.test_loader = DataLoader(train_dataset, batch_size=self.config.params_batch_size, shuffle=False, pin_memory=True)  # Assuming same dataset for simplicity

        # Model initialization (assuming a custom Model_Optimization class exists)
        # Load the trained U-Net model
        self.model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=3,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2
        )  #####.to(self.device)
        self.model.load_state_dict(torch.load(self.config.model_save_path))
        self.model.eval()

        # Loss and optimizer
        self.loss_function = nn.CrossEntropyLoss()
        if self.config.params_optimizer_name == "Adam":
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.config.params_learning_rate, weight_decay=self.config.params_weight_decay)
        elif self.config.params_optimizer_name == 'SGD':
            self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.config.params_learning_rate, weight_decay=self.config.params_weight_decay)
        else:
            self.optimizer = torch.optim.RMSprop(self.model.parameters(), lr=self.config.params_learning_rate, weight_decay=self.config.params_weight_decay)

    def train(self):
        for epoch in range(self.config.params_epochs):
            self.model.train()
            for batch_features, batch_labels, _ in self.train_loader:
                #### batch_features, batch_labels = batch_features.to(self.device), batch_labels.to(self.device)
                y_pred = self.model(batch_features)
                loss = self.loss_function(y_pred, batch_labels)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

    def evaluate(self):
        self.model.eval()
        total = 0
        correct = 0
        with torch.no_grad():
            for batch_features, batch_labels, _ in self.test_loader:
               ####  batch_features, batch_labels = batch_features.to(self.device), batch_labels.to(self.device)
                y_pred = self.model(batch_features)
                _, predicted = torch.max(y_pred, 1)
                total += batch_labels.size(0)
                correct += (predicted == batch_labels).sum().item()
        accuracy = correct / total
        return accuracy

    def save_best_model(self, accuracy):
        if not os.path.exists(self.config.best_model_path) or accuracy > self.best_accuracy:
            torch.save(self.model.state_dict(), self.config.best_model_path)
            self.best_accuracy = accuracy
            logger.info(f"Saved best model with accuracy {accuracy:.4f} to {self.config.best_model_path}")

In [11]:
def objective(trial):
    config = ConfigurationManager()
    config = config.get_model_optimization_config(trial)
    
    model_opt = ModelOptimization(config)
    model_opt.train()
    accuracy = model_opt.evaluate()
    
    # Save the best model
    model_opt.save_best_model(accuracy)
    
    return accuracy

In [12]:
try:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=5)  # Adjust number of trials as needed
    logger.info(f"Best trial: {study.best_trial.params}, Best accuracy: {study.best_value}")
except Exception as e:
    logger.error(f"Error during optimization: {str(e)}")
    raise e

[I 2025-03-27 18:48:54,983] A new study created in memory with name: no-name-2480e19f-e72e-42a6-ab41-e96b2a312492


[2025-03-27 18:48:54,991: INFO: common : yaml file: config/config.yaml loaded successfully]
[2025-03-27 18:48:54,995: INFO: common : yaml file: params.yaml loaded successfully]
[2025-03-27 18:48:54,996: INFO: common : created directory at: artifacts]
[2025-03-27 18:48:54,997: INFO: common : created directory at: artifacts/optimized_model]
[2025-03-27 18:48:55,000: INFO: 3460109881 : Found 20 files in artifacts/data_preprocessing/preprocessed]


/home/priyanshu1303d/anaconda3/envs/heart/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


[2025-03-27 18:50:44,486: INFO: 2962808286 : Saved best model with accuracy 1044390.8500 to artifacts/optimized_model/best_model.pth]


[I 2025-03-27 18:50:44,487] Trial 0 finished with value: 1044390.85 and parameters: {'epochs': 7, 'lr': 0.000157613418690593, 'batch_size': 128, 'weight_decay': 0.0007822573353391233, 'dropout_rate': 0.1, 'Optimizer_name': 'SGD'}. Best is trial 0 with value: 1044390.85.


[2025-03-27 18:50:44,492: INFO: common : yaml file: config/config.yaml loaded successfully]
[2025-03-27 18:50:44,494: INFO: common : yaml file: params.yaml loaded successfully]
[2025-03-27 18:50:44,495: INFO: common : created directory at: artifacts]
[2025-03-27 18:50:44,496: INFO: common : created directory at: artifacts/optimized_model]
[2025-03-27 18:50:44,498: INFO: 3460109881 : Found 20 files in artifacts/data_preprocessing/preprocessed]


[W 2025-03-27 18:51:18,250] Trial 1 failed with parameters: {'epochs': 6, 'lr': 0.000523082586818027, 'batch_size': 64, 'weight_decay': 0.013836228370503394, 'dropout_rate': 0.2, 'Optimizer_name': 'RMSprop'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/priyanshu1303d/anaconda3/envs/heart/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_196377/4076817030.py", line 6, in objective
    model_opt.train()
  File "/tmp/ipykernel_196377/2962808286.py", line 41, in train
    loss.backward()
  File "/home/priyanshu1303d/anaconda3/envs/heart/lib/python3.10/site-packages/torch/_tensor.py", line 626, in backward
    torch.autograd.backward(
  File "/home/priyanshu1303d/anaconda3/envs/heart/lib/python3.10/site-packages/torch/autograd/__init__.py", line 347, in backward
    _engine_run_backward(
  File "/home/priyanshu1303d/anaconda3/envs/heart/lib/python3

KeyboardInterrupt: 